In [1]:
import torch
#torch.set_default_dtype(torch.float64)
import numpy as np
import ase.io

from pytorch_prototype.code_pytorch import *
from pytorch_prototype.utilities import *
from pytorch_prototype.miscellaneous import ClebschGordan

from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species
from sklearn.kernel_ridge import KernelRidge

from matplotlib import pyplot as plt
import tqdm

In [2]:
METHANE_PATH = 'methane.extxyz'
train_slice = '0:400'
test_slice = '400:500'
L_MAX = 6
HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 15,
    'max_angular': L_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.3,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'

}
BATCH_SIZE = 1000
DEVICE = 'cuda'
clebsch = ClebschGordan(L_MAX)

In [3]:
def initialize_wigner_single(first, second):
    first_b_size, first_m_size = first.shape[0], first.shape[2]
    second_b_size, second_m_size = second.shape[0], second.shape[2]
    first = first.transpose(1, 2)
    second = second.transpose(1, 2)
    first = first.reshape([-1, first.shape[2]])
    second = second.reshape([-1, second.shape[2]])
    result = torch.matmul(first, second.transpose(0, 1))
    result = result.reshape(first_b_size, first_m_size, second_b_size, second_m_size)
    return result.transpose(1, 2)

def initialize_wigner(first, second):
    result = {}
    for key in first.keys():
        result[key] = initialize_wigner_single(first[key], second[key])
    return result

In [4]:
class WignerKernel(torch.nn.Module):
    def __init__(self, clebsch, lambda_max, num_iterations):
        super(WignerKernel, self).__init__()
        main = [WignerCombiningUnrolled(clebsch.precomputed_, lambda_max, algorithm = 'vectorized') 
                for _ in range(num_iterations)]
        self.main = nn.ModuleList(main)
        self.last = WignerCombiningUnrolled(clebsch.precomputed_, 0, algorithm = 'vectorized')
       
            
    def forward(self, X):
        wig_now = X
        for block in self.main:
            wig_now = block(wig_now, X)
        result = self.last(wig_now, X)
        return result['0'][:, 0, 0]

In [5]:
def compute_kernel(model, first, second, batch_size = 1000):
    wigner = initialize_wigner(first, second)
   
    for key in wigner.keys():
        initial_shape = [wigner[key].shape[0], wigner[key].shape[1]]
        wigner[key] = wigner[key].reshape([-1, wigner[key].shape[2], wigner[key].shape[3]])
    ''' for key in wigner.keys():
        print(key, wigner[key].shape)'''
    
    total = initial_shape[0] * initial_shape[1]
    result = []
    #print(total, batch_size)
    #print(initial_shape)
    for ind in tqdm.tqdm(range(0, total, batch_size)):
        now = {}
        for key in wigner.keys():
            now[key] = wigner[key][ind : ind + batch_size]
        result_now = model(now)
        result.append(result_now)
        
    result = torch.cat(result, dim = 0)
    return result.reshape(initial_shape)

In [6]:
train_structures = ase.io.read(METHANE_PATH, index = train_slice)
test_structures = ase.io.read(METHANE_PATH, index = test_slice)
all_species = get_all_species(train_structures + test_structures)

for struc in train_structures:
    mask_center_atoms_by_species(struc, species_select=["C"])
for struc in test_structures:
    mask_center_atoms_by_species(struc, species_select=["C"])


train_coefs = get_coefs(train_structures, HYPERS, all_species)
test_coefs = get_coefs(test_structures, HYPERS, all_species)
'''for key in train_coefs.keys():
    print(key, train_coefs[key].shape)'''

L2_mean = get_L2_mean(train_coefs)
#print(L2_mean)
for key in train_coefs.keys():
    train_coefs[key] /= np.sqrt(L2_mean)
    test_coefs[key] /= np.sqrt(L2_mean)

for key in train_coefs.keys():
    train_coefs[key] = train_coefs[key].to(DEVICE)
    test_coefs[key] = test_coefs[key].to(DEVICE)

model = WignerKernel(clebsch, L_MAX, 1)
model = model.to(DEVICE)

train_train_kernel = compute_kernel(model, train_coefs, train_coefs, batch_size = BATCH_SIZE)
train_test_kernel = compute_kernel(model, train_coefs, test_coefs, batch_size = BATCH_SIZE)

100%|██████████| 40/40 [00:04<00:00,  8.33it/s]


In [7]:
print(train_train_kernel.shape)
print(train_test_kernel.shape)

torch.Size([400, 400])
torch.Size([400, 100])
